In [29]:
import torch
import torch.nn as nn

In [30]:
device = torch.device("cuda")

In [31]:
class SelfAttention(nn.Module):
    def __init__(self, embed_size, heads):
        super(SelfAttention, self).__init__()
        self.embed_size = embed_size
        self.heads = heads
        self.head_dim = embed_size // heads

        #prevent embed_size cannot be integer divided by heads
        assert (self.head_dim * heads == embed_size), "Embed size needs to be div by heads"

        #linear lay for value key and queries
        self.values = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.keys = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.queries = nn.Linear(self.head_dim, self.head_dim, bias=False)

        #fully connected concatenated output
        self.fc_out = nn.Linear(heads * self.head_dim, embed_size)

    def forward(self, values, keys, query, mask):
        #number of training examples, how many examples we sent in at the same time
        #batch size
        N = query.shape[0]
        value_len, key_len, query_len = values.shape[1], keys.shape[1], query.shape[1]

        # splie embedding into self.heads pieces
        values = values.reshape(N, value_len, self.heads, self.head_dim)
        keys = keys.reshape(N, key_len, self.heads, self.head_dim)
        query = query.reshape(N, query_len, self.heads, self.head_dim)

        #send to the linear layer
        values = self.values(values)
        keys = self.keys(keys)
        query = self.queries(query)
        #multiply query with keys
        energy = torch.einsum("nqhd,nkhd->nhqk", [query, keys])
        #queries shape (N, query_len, heads, heads_dim)
        #keys shape (N, key_len, heads, heads_dim)
        # energy shape (N, heads, query_len, key_len)

        if mask is not None:
            #set masked to minus infinity, softmax will set it to zero
            energy = energy.masked_fill(mask == 0, float("-1e20"))

        #dim = 3, normalize across the key_len dimension
        attention = torch.softmax(energy / (self.embed_size ** (1/2)), dim = 3)


        out = torch.einsum("nhql,nlhd->nqhd", [attention, values]).reshape(
            N, query_len, self.heads * self.head_dim
        )
        #multiply atention with values
        #attention shape: (N, heads, query_len, key_len)
        #values shape: (N, value_len, heads, heads_dim)
        # after einsum (N, query_len, heads, heads_dim), then flatten last two dimensions

        out = self.fc_out(out)
        return out


In [32]:
class TransformerBlock(nn.Module):
    def __init__(self, embed_size, heads, dropout, forward_expansion):
        super(TransformerBlock, self).__init__()
        self.attention = SelfAttention(embed_size, heads)

        #similar to batchnorm, but take the average for every single example not for batch
        self.norm1 = nn.LayerNorm(embed_size)
        self.norm2 = nn.LayerNorm(embed_size)

        #expansion block, do some computations and map it back to the same dimension
        self.feed_forward = nn.Sequential(
            nn.Linear(embed_size, forward_expansion*embed_size),
            nn.ReLU(),
            nn.Linear(forward_expansion*embed_size, embed_size)
        )
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, value, key, query, mask):
        attention = self.attention(value, key, query, mask)

        #???? skip connection
        x =  self.dropout(self.norm1(attention + query))
        forward = self.feed_forward(x)
        out = self.dropout(self.norm2(forward + x))
        return out

In [33]:
class Encoder(nn.Module):
    def __init__(
            self,
            src_vocab_size,
            embed_size,
            num_layers,
            heads,
            device,
            forward_expansion,
            dropout,
            max_length, #??max sentence length, delete abnormal long sentences in the dataset
    ):
        super(Encoder, self).__init__()
        self.embed_size = embed_size
        self.device = device
        self.word_embedding = nn.Embedding(src_vocab_size, embed_size)
        self.position_embedding = nn.Embedding(max_length, embed_size)

        self.layers = nn.ModuleList(
            [
                TransformerBlock(
                    embed_size,
                    heads,
                    dropout=dropout,
                    forward_expansion=forward_expansion
                )
                for _ in range(num_layers)  
                
            ]
        )
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask):
        #N examples comes in, them some sequence length
        N, seq_length = x.shape
        #0,1,2,3,... for every example
        positions = torch.arange(0, seq_length).expand(N, seq_length).to(self.device)

        out = self.dropout(self.word_embedding(x) + self.position_embedding(positions))

        for layer in self.layers:
            #in encoder, key, value queries all going to be the same
            out = layer(out, out, out, mask)
        
        return out


In [34]:
class DecoderBlock(nn.Module):
    def __init__(self, embed_size, heads, forward_expansion, dropout, device):
        super(DecoderBlock, self).__init__()

        self.attention = SelfAttention(embed_size, heads)
        self.norm = nn.LayerNorm(embed_size)
        self.transformer_block = TransformerBlock(
            embed_size, heads, dropout, forward_expansion
        )
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x, value, key, src_mask, trg_mask):
        #source mask is "optional", trg_mask is padding
        attention = self.attention(x, x, x, trg_mask)
        query = self.dropout(self.norm(attention + x)) #with skip connection
        out = self.transformer_block(value, key, query, src_mask)
        return out



In [35]:
class Decoder(nn.Module):
    def __init__(self,
                 trg_vocab_size,
                 embed_size,
                 num_layers,
                 heads,
                 forward_expansion,
                 dropout,
                 device,
                 max_length):
        super(Decoder, self).__init__()
        self.device = device
        self.word_embedding = nn.Embedding(trg_vocab_size, embed_size)
        self.position_embedding = nn.Embedding(max_length, embed_size)

        self.layers = nn.ModuleList(
            [DecoderBlock(embed_size, heads, forward_expansion, dropout, device)
             for _ in range(num_layers)]
        )

        self.fc_out = nn.Linear(embed_size, trg_vocab_size)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x, enc_out, src_mask, trg_mask):
        N, seq_length = x.shape
        positions = torch.arange(0, seq_length).expand(N, seq_length).to(self.device)
        x = self.dropout((self.word_embedding(x)+ self.position_embedding(positions)))

        for layer in self.layers:
            x = layer(x, enc_out, enc_out, src_mask, trg_mask)
        
        out = self.fc_out(x)
        return out

In [36]:
class Transformer(nn.Module):
    def __init__(
            self,
            src_vocab_size,
            trg_vocab_size,
            src_pad_idx,
            trg_pad_idx,
            embed_size = 256,
            num_layers = 6,
            forward_expansion = 4,
            heads = 8,
            dropout = 0,
            device = "cuda",
            max_length = 100
    ):
        super(Transformer, self).__init__()

        self.encoder = Encoder(
            src_vocab_size,
            embed_size,
            num_layers,
            heads,
            device,
            forward_expansion,
            dropout,
            max_length
        )

        self.decoder = Decoder(
            trg_vocab_size,
            embed_size,
            num_layers,
            heads,
            forward_expansion,
            dropout,
            device,
            max_length
        )

        self.src_pad_idx = src_pad_idx
        self.trg_pad_idx = trg_pad_idx
        self.device = device
    
    def make_src_mask(self, src):
        src_mask = (src != self.src_pad_idx).unsqueeze(1).unsqueeze(2)
        # shape of  (N,1,1,src_len)
        return src_mask.to(self.device)
    
    def make_trg_mask(self, trg):
        N, trg_len = trg.shape
        trg_mask = torch.tril(torch.ones((trg_len, trg_len))).expand(
            N, 1, trg_len, trg_len
        )
        return trg_mask.to(self.device)
    
    def forward(self, src, trg):
        src_mask = self.make_src_mask(src)
        trg_mask = self.make_trg_mask(trg)
        enc_src = self.encoder(src, src_mask)
        out = self.decoder(trg, enc_src, src_mask, trg_mask)
        return out

In [37]:
x = torch.tensor([[1,5,6,4,3,9,5,2,0], [1,8,7,3,4,5,6,7,2]]).to(device)
trg = torch.tensor([[1,7,4,3,5,9,2,0],[1,5,6,2,4,7,6,2]]).to(device)

src_pad_idx = 0
trg_pad_idx = 0
src_vocab_size = 10
trg_vocab_size = 10
model = Transformer(src_vocab_size, trg_vocab_size, src_pad_idx, trg_pad_idx).to(device)
out = model(x, trg[:, :-1])

In [38]:
out.shape

torch.Size([2, 7, 10])

In [39]:
out

tensor([[[-7.7819e-01, -3.3744e-01, -2.6475e-02, -8.1612e-01,  2.0964e-01,
          -2.0886e-01,  1.1413e+00, -1.4074e-01, -7.9480e-01,  1.5883e-01],
         [-3.7552e-01,  1.0055e-01,  1.7208e-01, -6.3905e-01, -2.7463e-01,
           6.3939e-02,  1.0818e+00,  3.6090e-01, -8.8235e-01, -4.0167e-01],
         [-4.2685e-01, -2.7327e-01,  4.7389e-01, -8.6248e-01,  2.8796e-01,
          -2.3583e-01,  6.7756e-01,  9.3188e-01,  1.9699e-03,  3.9210e-01],
         [-7.5244e-01,  1.4607e-01, -7.4415e-02, -6.0634e-01,  6.4295e-01,
          -9.2918e-01,  2.7038e-01,  5.1316e-01, -8.5358e-01, -4.5142e-01],
         [ 3.9938e-01,  6.4404e-01,  5.0709e-01, -4.3246e-03,  6.6678e-01,
          -3.3545e-01,  6.8084e-01,  6.4793e-01, -1.3101e-01, -2.0912e-01],
         [ 8.3105e-02,  4.7657e-01,  2.6090e-01, -8.6656e-01,  3.0147e-01,
          -1.0521e+00,  1.9610e-01,  1.1325e+00, -6.0224e-01,  5.4821e-01],
         [ 7.0944e-01, -2.3330e-01,  3.5842e-01, -7.1632e-01,  1.0577e+00,
           4.4665e-